#Demo - Predicting Face Touches
##“Automatic Detection of Face Touch and Self-adaptors in Infants”

This library uses some capabilities from Chambers et al. study and their repositories for infant skeleton tracking.

- https://github.com/cchamber/Infant_movement_assessment
- Chambers, Claire; Seethapathi, Nidhi; Saluja, Rachit; Johnson, Michelle; Kording, Konrad Paul (2019): Computer vision to automatically assess infant neuromotor risk. figshare. Dataset. https://doi.org/10.6084/m9.figshare.8161430.v5 

##<b>Part 0: Download dependencies and library</b>

###1. Download Git repository, create required folders and mount drive.

In [ ]:
!git clone https://github.com/bt403/openpose-research-keras
!mkdir /content/openpose-research-keras/model
from google.colab import drive
drive.mount('/content/drive')

###2. Download library dependencies for OpenPose

In [ ]:
#@title RUN Download dependencies
!apt-get install ffmpeg --yes
!pip install ConfigObj
!pip install sk-video
!pip install tqdm
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
!pip install mediapipe
!pip install face-alignment	
!pip install arff
!pip install scikit-multilearn

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
if (len(GPUs) > 0):
  gpu = GPUs[0]
  def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  printm()


##<b>Part I: Export frames and pose from videos</b>

###1. Place videos in sample_videos folder
The videos that need to be processed should be placed in the "sample_videos" folder. In this case, the videos are copied from Drive. 

In [ ]:
%cp /content/drive/MyDrive/ResearchProject/videos-mp4/* /content/openpose-research-keras/sample_videos/

Assuming there are videos inside the folder, the following steps can be run.

###2. Execute command to export frames from videos.
The following script exports frames from videos for tagging and training. The command exports 3 frames per second. The main parameters are the following:

1.   <b>videos_path</b>: Path where the videos are located. By default it loads videos from ./sample_videos .
2.   <b>export_images_path</b>: Path where the videos will be saved



In [ ]:
%cd /content/openpose-research-keras/
!CUDA_VISIBLE_DEVICES=0 python3 /content/openpose-research-keras/export_frames.py --videos_path './sample_videos' --export_images_path '/content/drive/MyDrive/ResearchProject/exported_frames/'

###3. Download and copy OpenPose infant model to folder
The following script downloads Chambers et al. model from their open repository.

- Chambers, Claire; Seethapathi, Nidhi; Saluja, Rachit; Johnson, Michelle; Kording, Konrad Paul (2019): Computer vision to automatically assess infant neuromotor risk. figshare. Dataset. https://doi.org/10.6084/m9.figshare.8161430.v5 

In [ ]:
%cd /content/
!wget https://figshare.com/ndownloader/articles/8161430?private_link=10034c230ad9b2b2a6a4
!unzip "/content/8161430?private_link=10034c230ad9b2b2a6a4"
!unzip pose_extraction.zip
%cd /content/openpose-research-keras/
%cp -a /content/pose_extraction/colab_openpose/models/model.h5 /content/openpose-research-keras/model/model.h5

###4. Execute command to export poses from videos.
The following script exports the infant pose data from videos. The main parameters are the following:

1.   <b>videos_path</b>: Path where the videos are located. By default it loads videos from ./sample_videos
2.   <b>model_path</b>: Path where the openpose model is located. By default it loads from ./model/model.h5.

In [ ]:
%cd /content/openpose-research-keras/
!CUDA_VISIBLE_DEVICES=0 python3 /content/openpose-research-keras/export_openpose_values.py --videos_path './sample_videos' --model_path './model/model.h5'

###5. Copy exported raw coordinates to data folder.

In [12]:
%mkdir /content/openpose-research-keras/data
%mkdir /content/openpose-research-keras/data/processed
%mkdir /content/openpose-research-keras/data/estimates
%cp  /content/openpose-research-keras/sample_videos/* /content/openpose-research-keras/data/processed

###6. Run processing of exported coordinates
The following script processes the coordinates of the videos by selecting limbs, smoothing, normalising and interpolating. The main parameters are the following:

1.   <b>input_path</b>: Path where the raw coordinates and videos are located. By default it loads the input from ./data/processed/
2.   <b>output_path</b>: Path where the new processed estimates are saved. By default it saves them to: ./data/estimates/

In [ ]:
%cd /content/openpose-research-keras/
!CUDA_VISIBLE_DEVICES=0 python3 /content/openpose-research-keras/process_pose_values.py --input_path './data/processed/' --output_path './data/estimates/'

##<b>Part II: Export aggregated features</b>


### 1. Requirements

This section requires the following inputs:
1. <b>Generated pose estimates</b>: Ensure the output coordinates generated in Part I are located in the ./data/estimates/ coordinates.
2. <b>Generated image frames</b>: Path for exported image frames.
3. <b>CSV file with tagged data</b>: The CSV file should contain the following columns: 
- name_image: {video_name}\_{video_id}\_{frame_num_in_video}\_{frame_id}. Example: video_1_1001_121.jpg. These should match the extracted frames.
- head_area: on-head or outside-head depending if the infant is touching his face.
- ears: 1 or 0. Depending if the infant is touching this location.
- neck: 1 or 0. Depending if the infant is touching this location.	
- mouth: 1 or 0. Depending if the infant is touching this location.
- cheeks: 1 or 0. Depending if the infant is touching this location.	
- eyes: 1 or 0. Depending if the infant is touching this location.
- nose: 1 or 0. Depending if the infant is touching this location.	
- forehead: 1 or 0. Depending if the infant is touching this location.

The number of rows of the CSV file should match the number of image frames found in the path where the generated image frames are located. In this case the following script copies the file with the tagged image data from Drive.

In [20]:
!cp /content/drive/MyDrive/ResearchProject/videos-processed/data_full_csv.csv /content/openpose-research-keras/

### 2. Execute extraction of aggregated features
The following command executes the script to export the processed values. This command will generate and save the distance and angular features. It also will extract the face and wrist region of the infants in each frame. The required arguments are the following:
1.   <b>exported_frames</b>: Path where the exported frames are located. By default it loads the frames from /content/drive/MyDrive/ResearchProject/exported_frames/
2.   <b>cropped_output_path</b>: Path where cropped image frames will be saved. By default it saves the output images to /content/drive/MyDrive/ResearchProject/cropped/
3.   <b>csv_data</b>: Path of CSV file described above. By default it loads from /content/openpose-research-keras/data_full_csv.csv
4. <b>pose_estimates_path</b>: Path of generated pose estimates. By default it loads from /content/openpose-research-keras/data_full_csv.csv
5. <b>output_path</b>: Path to save generated features. By defaults it saves them to /content/openpose-research-keras/data/estimates/processed_features.pkl


In [ ]:
%cd /content/openpose-research-keras/
!CUDA_VISIBLE_DEVICES=0 python3 /content/openpose-research-keras/process_feature_extraction.py --exported_frames '/content/drive/MyDrive/ResearchProject/exported_frames/'  --cropped_output_path '/content/drive/MyDrive/ResearchProject/cropped3/' --csv_data '/content/openpose-research-keras/data_full_csv_v2.csv'  --pose_estimates '/content/openpose-research-keras/data/estimates/'

### 3. Execute extraction of smaller face areas and confidence scores
After obtaining the aggregated features, the following code will obtain the exported exported mouth area, nose area and their confidence scores using 3D-FAN. This script requires the following parameters as input:

1.   <b>processed_features</b>: Path to the exported processed features. By default the features were exported to './data/estimates/processed_features.pkl'.
2.   <b>exported_frames</b>: Path where the exported frames are located. By default it loads the frames from /content/drive/MyDrive/ResearchProject/exported_frames/
3.   <b>eyes_path</b>: Path where the exported cropped eyes region will be saved. By default it saves the images to: /content/drive/MyDrive/ResearchProject/mouth/
4. <b>mouth_path</b>: Path where the exported cropped mouth region will be saved. By default it saves the images to: /content/drive/MyDrive/ResearchProject/mouth/

The code will save the updated processed_features in the same path.

In [ ]:
%cd /content/openpose-research-keras/
!CUDA_VISIBLE_DEVICES=0 python3 /content/openpose-research-keras/process_face_region.py --processed_features './data/estimates/processed_features.pkl' --exported_frames '/content/drive/MyDrive/ResearchProject/exported_frames/' --eyes_path '/content/drive/MyDrive/ResearchProject/eyes/' --mouth_path '/content/drive/MyDrive/ResearchProject/mouth/'

###4. Extract HOG Features
Based on the obtained images for eyes, mouth and face, the HOG features can be extracted. 

The following code extracts the HOG features of these regions and saves them. The code receives the following inputs:
1.   <b>processed_features</b>: Path to the exported processed features. By default the features were exported to './data/estimates/processed_features.pkl'.
2.   <b>output_path</b>: Path where the HOG features will be saved. By default it saves them to ./data/estimates/
2.   <b>eyes_folder</b>: Path where the eyes area images were saved. By default it loads them from /content/drive/MyDrive/ResearchProject/eyes/ .
2.   <b>mouth_folder</b>: Path where the mouth area images were saved. By default it loads them from /content/drive/MyDrive/ResearchProject/mouth/ .

In [ ]:
%cd /content/openpose-research-keras/
!CUDA_VISIBLE_DEVICES=0 python3 /content/openpose-research-keras/extract_hog_features.py

##Part III: Preprocess and augment features

This section performs final preprocessing and augmentation before running the classification algorithms.

1.   <b>processed_features</b>: Path to the exported processed features. By default the features were exported to './data/estimates/processed_features.pkl'.
2.   <b>final_features</b>: Path where the final features will be saved after preprocessing. By default it saves them to ./data/estimates/final_features.pkl

In [ ]:
%cd /content/openpose-research-keras/
!CUDA_VISIBLE_DEVICES=0 python3 /content/openpose-research-keras/preprocess_features.py 

##Part IV: Train Classifier - RF - PCA - SVM Model
This section trains the classifiers based on the final preprocessed data. However, before training the classifier, the data should be divided into training and testing. However this is dependant on the problem so I will show an example based on sample data, but it could vary depending on the context.

### 1. Divide train and test data

The following code loads the preprocessed data and divides it into training and testing sets. Then it saves them to a pickle file.

In [175]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, GroupKFold, cross_val_score, cross_validate
pdata = pd.read_pickle('./data/estimates/final_features.pkl')
augmented = True

if augmented:
  pdata_augmented = pd.read_pickle('./data/estimates/final_features_augmented.pkl')

splitter = GroupShuffleSplit(test_size=.5, n_splits=2, random_state = 1)
split = splitter.split(pdata, groups=pdata['video_name'])
train_inds, test_inds = next(split)
train = pdata.iloc[train_inds].copy()
test = pdata.iloc[test_inds].copy()

if (augmented):
  train_augmented = pdata_augmented.iloc[train_inds].copy()
  train = pd.concat([train, train_augmented])

train.to_pickle('./data/estimates/train_data.pkl')
test.to_pickle('./data/estimates/test_data.pkl')

### 2. Train classifier

The following scripts performs Feature Selection with Random Forest, Feature Reduction with PCA and Classification with SVM. Also, it cross-validates with 5 folds and does a GridSearch to look for the best hyperparameters in the training set. The pipeline also standardises the values and fills any remaining missing value based on the training set.

The script receives as input the following parameters:
1.   <b>train_data</b>: Path to pkl file with train data. By default it loads the file from './data/estimates/train_data.pkl'.
2.   <b>test_data</b>: Path to pkl file with test data. By default it loads the file from './data/estimates/test_data.pkl'.

In [ ]:
%cd /content/openpose-research-keras/
!CUDA_VISIBLE_DEVICES=0 python3 /content/openpose-research-keras/train_classifier_rf_pca_svm.py 

##Part V: Train Classifier - Autoencoder Model

### 1. Divide train and test data

The following code loads a sample preprocessed data and divides it into training and testing sets. Then it saves them to a pickle file.

In [205]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, GroupKFold, cross_val_score, cross_validate
pdata = pd.read_pickle('./data/estimates/final_features.pkl')
augmented = True

if augmented:
  pdata_augmented = pd.read_pickle('./data/estimates/final_features_augmented.pkl')

splitter = GroupShuffleSplit(test_size=.5, n_splits=2, random_state = 1)
split = splitter.split(pdata, groups=pdata['video_name'])
train_inds, test_inds = next(split)
train = pdata.iloc[train_inds].copy()
test = pdata.iloc[test_inds].copy()

if (augmented):
  train_augmented = pdata_augmented.iloc[train_inds].copy()
  train = pd.concat([train, train_augmented])

train.to_pickle('./data/estimates/train_data.pkl')
test.to_pickle('./data/estimates/test_data.pkl')

hog_features_face = pd.read_pickle('./data/estimates/hog_features_face.pkl')
hog_features_eyes = pd.read_pickle('./data/estimates/hog_features_eyes.pkl')
hog_features_mouth = pd.read_pickle('./data/estimates/hog_features_mouth.pkl')
hog_features_face_flipped = pd.read_pickle('./data/estimates/hog_features_face_flipped.pkl')
hog_features_eyes_flipped = pd.read_pickle('./data/estimates/hog_features_eyes_flipped.pkl')
hog_features_mouth_flipped = pd.read_pickle('./data/estimates/hog_features_mouth_flipped.pkl')
hog_features = pd.concat((hog_features_face, hog_features_eyes, hog_features_mouth), axis=1)
hog_features_flipped = pd.concat((hog_features_face, hog_features_eyes, hog_features_mouth), axis=1)

hog_features_train = hog_features.iloc[train_inds]
hog_features_train_flipped = hog_features_flipped.iloc[train_inds]
hog_features_train = pd.concat([hog_features_train, hog_features_train_flipped])
hog_features_test = hog_features.iloc[test_inds]

hog_features_train.to_pickle('./data/estimates/train_data_hog.pkl')
hog_features_test.to_pickle('./data/estimates/test_data_hog.pkl')

### 2. Train classifier

The following scripts performs the training for the Autoencoders with SVM classifier. Also, it cross-validates with 5 folds and does a GridSearch to look for the best hyperparameters in the training set. The pipeline also standardises the values and fills any remaining missing value based on the training set.

The script receives as input the following parameters:
1.   <b>train_data</b>: Path to pkl file with train data. By default it loads the file from './data/estimates/train_data.pkl'.
2.   <b>test_data</b>: Path to pkl file with test data. By default it loads the file from './data/estimates/test_data.pkl'.
3.   <b>train_data_hog</b>: Path to pkl file with train hog data. By default it loads the file from './data/estimates/train_data_hog.pkl'.
4.   <b>test_data_hog</b>: Path to pkl file with test hog data. By default it loads the file from './data/estimates/test_data_hog.pkl'.

In [ ]:
%cd /content/openpose-research-keras/
!CUDA_VISIBLE_DEVICES=0 python3 /content/openpose-research-keras/train_classifier_autoencoder.py 